## IMPORTANTE: Reiniciar Kernel

Se você estiver encontrando erro `OSError: could not get source code`, **reinicie o kernel do notebook** (Ctrl+Shift+P → "Restart Kernel") antes de executar as células abaixo.

## 📦 Setup

In [1]:
# 1. Carregar dados + 
import sys
sys.path.append('..')  # Adiciona o diretório pai
from src.utils.io.io_local import *
from src.utils.io.io_clearml import *

from src.utils.io import load_dataframe
from config import config_custom as config
from src.pipelines.pipeline_processamento import executar_pipeline_processamento
from src.pipelines.pipeline_features import executar_pipeline_features
from src.pipelines import treinar_pipeline_completo, treinar_rapido

df_raw = load_dataframe('../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv')
print(f'Dados brutos: {df_raw.shape}')

Dados brutos: (1720, 40)


# Pipelines

## Pipeline processamento

In [ ]:
# 1. Carregar dados + # 2. PROCESSAMENTO (Limpeza + Imputação)
df_raw = load_dataframe('../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv')
print(f'Dados brutos: {df_raw.shape}')

# 2. PROCESSAMENTO (Limpeza + Imputação)
df_proc = executar_pipeline_processamento(
    df_raw,
    config_imputacao_customizada=config.CONFIG_IMPUTACAO_CUSTOMIZADA,
    criar_agrupamento_temporal=True,
    nome_coluna_agrupamento='mes-ano'
)
print(f'Após processamento: {df_proc.shape}')
print(f'NAs restantes: {df_proc.isna().sum().sum()}')

## Pipeline features

In [ ]:
# 3. FEATURES (Codificação + Derivadas + Normalização)
df_feat, artefatos = executar_pipeline_features(
    df_proc,
    # Codificação
    aplicar_codificacao=True,
    metodo_codificacao='label',           # 'label' ou 'onehot'
    sufixo_codificacao='_cod',
    
    # Features derivadas
    criar_features_derivadas=True,
    tipos_features_derivadas=[
        'imc',                            # Índice de Massa Corporal
        'imc_classe',                     # Classe do IMC
        'heat_index',                     # Índice de calor
        'dew_point',                      # Ponto de orvalho
        't*u',                            # Temperatura × Umidade
    ],
    
    # Normalização
    aplicar_normalizacao=True,
    metodo_normalizacao='standard',       # 'standard', 'minmax', 'robust'
    agrupamento_normalizacao='mes-ano',   # Normalizar por grupo
    sufixo_normalizacao='_norm',
)

print(f'Após features: {df_feat.shape}')
print(f'Artefatos criados: {list(artefatos.keys())}')
df_feat.to_csv("../dados/resultados/dados_processados_novas_features.csv")
# Visualizar resultado
df_feat.head()


# Pipeline de treinamento

In [ ]:
from config.config_gerais import PARAMS_PADRAO

# Definir coluna alvo e features
coluna_alvo = 'p1'

# Usar apenas features principais para teste
features_treino = [
    'idade', 'sexo_cod', 'peso', 'altura',
    'tmedia', 'ur', 'vel_vento',
]

tipos_modelos ='regressao'

# Filtrar features que existem no DataFrame
features_existentes = [f for f in features_treino if f in df_feat.columns]
print(f"Features para treinamento: {features_existentes}")

# Preparar dados
df_treino = df_feat[features_existentes + [coluna_alvo]].dropna()
print(f"Dataset de treino: {df_treino.shape}")

# 4. TREINAMENTO
print("\n" + "="*60)
print("🤖 INICIANDO PIPELINE DE TREINAMENTO")
print("="*60)

resultado = treinar_pipeline_completo(
    dados=df_treino,
    coluna_alvo=coluna_alvo,
    tipo_problema=tipos_modelos,  # 'classificacao' ou 'regressao'
    params_setup=PARAMS_PADRAO,
    n_modelos_comparar=3,           # Testar top 3 modelos
    otimizar_hiperparametros=True,         # Otimizar hiperparâmetros
    n_iter_otimizacao=10,  # 10 iterações de otimização
    salvar_modelo_final=True,           # Salvar modelo
    nome_modelo="modelo_conforto_termico",
    pasta_modelos="modelos"
)

# Visualizar resultados
print("\n" + "="*60)
print("📊 RESULTADOS DO TREINAMENTO")
print("="*60)

# Nome do melhor modelo
nome_modelo = str(resultado['tabela_comparacao'].index[0])
print(f"\n✓ Melhor modelo: {nome_modelo}")

print(f"\n📈 Métricas principais:")
metricas = resultado['metricas_melhor']
for nome, valor in metricas.items():
    if isinstance(valor, (int, float)):
        print(f"  • {nome}: {valor:.4f}")

print(f"\n💾 Modelo salvo em: {resultado.get('caminho_modelo', 'N/A')}")

print("\n📋 Comparação de modelos (top 5 métricas):")
print(resultado['tabela_comparacao'].head())

In [ ]:
resultados_10_experimentos = {}
for i in range(10):
    print("\n" + "="*60)
    print(f"🤖 INICIANDO EXPERIMENTO {i+1}/10")
    print("="*60)
    
    resultado = treinar_pipeline_completo(
    dados=df_treino,
    coluna_alvo=coluna_alvo,
    tipo_problema='regressao',  # 'classificacao' ou 'regressao'
    params_setup=PARAMS_PADRAO,
    n_modelos_comparar=3,           # Testar top 3 modelos
    otimizar_hiperparametros=True,         # Otimizar hiperparâmetros
    n_iter_otimizacao=10,  # 10 iterações de otimização
    salvar_modelo_final=True,           # Salvar modelo
    nome_modelo="modelo_conforto_termico",
    pasta_modelos="modelos"
)
    
    resultados_10_experimentos[f'Experimento_{i+1}'] = resultado


In [ ]:
tabelas_comparacao = [dicionario["tabela_comparacao"] for dicionario in resultados_10_experimentos.values()] 
tabelas_comparacao =  [tabela.rename(columns={'Model':'Modelos','Accuracy': 'Acurácia', 'AUC': 'AUC', 'Recall': 'Recall', 'Prec.': 'Prec.', 'F1': 'F1'}) for tabela in tabelas_comparacao]
serie_nomes_modelos = tabelas_comparacao[0]['Modelos']
tabelas_comparacao = [tabela.select_dtypes(include='number') for tabela in tabelas_comparacao] 

In [ ]:

df_desvio_metricas= pd.concat(tabelas_comparacao).groupby(level=0).std()
df_media_metricas = pd.concat(tabelas_comparacao).groupby(level=0).mean()
df_media_desvio_metricas_str = df_media_metricas.round(2).astype(str) + " ± " + df_desvio_metricas.round(2).astype(str)
df_media_desvio_metricas_str = df_media_desvio_metricas_str.merge(serie_nomes_modelos, left_index=True, right_index=True)
df_media_metricas = df_media_metricas.merge(serie_nomes_modelos, left_index=True, right_index=True)
df_desvio_metricas = df_desvio_metricas.merge(serie_nomes_modelos, left_index=True, right_index=True)
df_desvio_metricas.to_csv('../dados/resultados/desvio_metricas.csv')
df_media_metricas.to_csv('../dados/resultados/media_metricas.csv')
df_media_desvio_metricas_str.to_csv('../dados/resultados/media_e_desvio_metricas_str.csv')

In [ ]:
df_media_desvio_metricas_str


# ClearML

## Pipeline Completo - Versão Local

**Por que não usar os decorators ClearML aqui?**

Os decorators `@pipeline` e `@component` do ClearML **sempre** tentam criar a infraestrutura de pipeline (tasks, serialização, etc.), mesmo quando configurado para `run_locally=True`. Isso causa erros ao tentar serializar DataFrames grandes e criar tasks automaticamente.

**Solução**: Para execução realmente local (sem servidor ClearML), usamos diretamente as funções originais dos pipelines **SEM** os decorators:
- `executar_pipeline_processamento()` - da pasta src/pipelines
- `treinar_pipeline_completo()` - da pasta src/treinamento

Isso funciona perfeitamente porque são as mesmas funções que você já usa normalmente, apenas sem a camada ClearML em cima.

**Quando usar os pipelines com decorators ClearML?**
- Quando você tem um servidor ClearML configurado
- Quando quer enviar jobs para execução remota
- Quando precisa de rastreamento completo no servidor ClearML

In [ ]:
# =========================================================================
# PIPELINE CLEARML COMPLETO - VERSÃO LOCAL SIMPLIFICADA
# =========================================================================
# NOTA: Os decorators @pipeline do ClearML sempre tentam criar infraestrutura,
#       mesmo com run_locally=True. Para execução realmente local, usamos as
#       funções wrapped (sem decorators) ou reimplementamos o fluxo.
# =========================================================================
import sys
sys.path.append('..')

# Imports das pipelines originais (SEM decorators ClearML)
from src.pipelines.pipeline_processamento import executar_pipeline_processamento
from src.pipelines.pipeline_treinamento_unified import treinar_pipeline_completo

print("="*70)
print("PIPELINE COMPLETO - VERSAO LOCAL (SEM CLEARML)")
print("="*70)

# 1. Verificar dados
print(f"\n1. Dados brutos: {df_raw.shape}")

# 2. PROCESSAMENTO
print(f"\n2. Executando processamento...")
df_processado = executar_pipeline_processamento(df_raw)
print(f"   Dados processados: {df_processado.shape}")

# 3. Preparar dados para treino (remover NAs)
print(f"\n3. Preparando dados para treino...")
print(f"   Dados limpos: {df_treino.shape}")

# 4. TREINAMENTO
print(f"\n4. Executando treinamento...")
print(f"   Coluna alvo: {coluna_alvo}")
print(f"   Tipo: regressao")

resultado = treinar_pipeline_completo(
    dados=df_treino,
    coluna_alvo=coluna_alvo,
    tipo_problema='regressao',
    n_modelos_comparar=2,  # Apenas 2 modelos para teste rapido
    otimizar_hiperparametros=False,  # Sem otimizacao para ser rapido
    salvar_modelo_final=False,
    nome_modelo="teste_clearml"
)

# 5. RESULTADOS
print("\n" + "="*70)
print("RESULTADOS")
print("="*70)

melhor_modelo = type(resultado['melhor_modelo']).__name__
print(f"\nMelhor modelo: {melhor_modelo}")

print(f"\nMetricas:")
metricas = resultado['metricas_melhor']
for nome, valor in list(metricas.items())[:5]:  # Primeiras 5 metricas
    if isinstance(valor, (int, float)):
        print(f"  {nome}: {valor:.4f}")

print(f"\nComparacao de modelos:")
print(resultado['tabela_comparacao'][['MAE', 'MSE', 'RMSE', 'R2']].head())

print("\n" + "="*70)
print("PIPELINE CONCLUIDO COM SUCESSO!")
print("="*70)

## Pipeline Completo COM ClearML

Agora vamos usar o ClearML **completo** para:
1. Criar **tasks separadas** para cada etapa
2. **Upload de datasets** (bruto e processado) como versões
3. **Registro de métricas** em cada pipeline
4. **Upload do modelo final**
5. **Artefatos**: tabelas de comparação, plots, etc.

**Pré-requisitos**: 
- ClearML configurado (`clearml-init`)
- Servidor ClearML acessível (pode ser demo.clear.ml)

In [ ]:
# =========================================================================
# PIPELINE COMPLETO COM CLEARML - REGISTRO TOTAL
# =========================================================================
import sys
sys.path.append('..')

from clearml import Task, Dataset, OutputModel
import pandas as pd
from pathlib import Path

# Imports dos pipelines
from src.pipelines.pipeline_processamento import executar_pipeline_processamento
from src.pipelines.pipeline_treinamento_unified import treinar_pipeline_completo

# Configuração
PROJECT_NAME = "conforto_termico"
coluna_alvo = 'p1'

print("="*80)
print("PIPELINE COMPLETO COM CLEARML - FULL TRACKING")
print("="*80)

# ============================================================================
# ETAPA 1: UPLOAD DE DADOS BRUTOS
# ============================================================================
print("\n[ETAPA 1] Upload de Dados Brutos para ClearML")
print("-"*80)

# Criar dataset de dados brutos
dataset_bruto = Dataset.create(
    dataset_name="dados_brutos_conforto_termico",
    dataset_project=PROJECT_NAME,
    description="Dados brutos de conforto termico de Santa Maria"
)

# Salvar temporariamente para upload
temp_path = Path("../dados/temp_clearml")
temp_path.mkdir(exist_ok=True)
arquivo_bruto = temp_path / "dados_brutos.csv"
df_raw.to_csv(arquivo_bruto, index=False)

# Adicionar arquivo ao dataset
dataset_bruto.add_files(str(arquivo_bruto))
dataset_bruto.upload()
dataset_bruto.finalize()

print(f"Dataset bruto criado: ID = {dataset_bruto.id}")
print(f"  - Shape: {df_raw.shape}")
print(f"  - Colunas: {len(df_raw.columns)}")

# ============================================================================
# ETAPA 2: PIPELINE DE PROCESSAMENTO
# ============================================================================
print("\n[ETAPA 2] Pipeline de Processamento")
print("-"*80)

# Criar task de processamento (COM FIX PARA NOTEBOOKS)
task_processamento = Task.init(
    project_name=PROJECT_NAME,
    task_name="Pipeline_Processamento",
    task_type=Task.TaskTypes.data_processing,
    reuse_last_task_id=False,  # Não reusar task anterior
    continue_last_task=False,   # Criar nova task sempre
    auto_connect_frameworks=False,  # Desabilita captura automática
    auto_resource_monitoring=False  # Desabilita monitoramento de recursos
)

# Conectar dataset de entrada
task_processamento.connect_configuration({
    "dataset_id": dataset_bruto.id,
    "dataset_name": "dados_brutos_conforto_termico"
})

# Executar processamento
print("Executando processamento...")
df_processado = executar_pipeline_processamento(df_raw)

# Registrar métricas de processamento
task_processamento.get_logger().report_single_value("linhas_entrada", df_raw.shape[0])
task_processamento.get_logger().report_single_value("linhas_saida", df_processado.shape[0])
task_processamento.get_logger().report_single_value("colunas_entrada", df_raw.shape[1])
task_processamento.get_logger().report_single_value("colunas_saida", df_processado.shape[1])
task_processamento.get_logger().report_single_value("nas_removidos", df_raw.isna().sum().sum() - df_processado.isna().sum().sum())

print(f"Dados processados: {df_processado.shape}")
print(f"NAs restantes: {df_processado.isna().sum().sum()}")

# Upload de dados processados como dataset
dataset_processado = Dataset.create(
    dataset_name="dados_processados_conforto_termico",
    dataset_project=PROJECT_NAME,
    parent_datasets=[dataset_bruto.id],
    description="Dados processados (sem NAs, limpeza aplicada)"
)

arquivo_processado = temp_path / "dados_processados.csv"
df_processado.to_csv(arquivo_processado, index=False)
dataset_processado.add_files(str(arquivo_processado))
dataset_processado.upload()
dataset_processado.finalize()

print(f"Dataset processado criado: ID = {dataset_processado.id}")

# Registrar como artefato na task
task_processamento.upload_artifact(
    "dados_processados_sample",
    artifact_object=df_processado.head(100)  # Apenas amostra
)

task_processamento.close()
print("Task de processamento finalizada!")

# ============================================================================
# ETAPA 3: PIPELINE DE TREINAMENTO
# ============================================================================
print("\n[ETAPA 3] Pipeline de Treinamento")
print("-"*80)

# Criar task de treinamento (COM FIX PARA NOTEBOOKS)
task_treinamento = Task.init(
    project_name=PROJECT_NAME,
    task_name="Pipeline_Treinamento",
    task_type=Task.TaskTypes.training,
    reuse_last_task_id=False,
    continue_last_task=False,
    auto_connect_frameworks=False,
    auto_resource_monitoring=False
)

# Conectar dataset processado
task_treinamento.connect_configuration({
    "dataset_id": dataset_processado.id,
    "coluna_alvo": coluna_alvo,
    "tipo_problema": "regressao"
})

# Preparar dados
df_treino = df_processado.dropna()
print(f"Dados para treino: {df_treino.shape}")

# Executar treinamento
print("Executando treinamento...")
resultado = treinar_pipeline_completo(
    dados=df_treino,
    coluna_alvo=coluna_alvo,
    tipo_problema='regressao',
    n_modelos_comparar=3,
    otimizar_hiperparametros=True,
    n_iter_otimizacao=10,
    salvar_modelo_final=True,
    nome_modelo="modelo_conforto_termico_clearml",
    pasta_modelos="../modelos"
)

# ============================================================================
# ETAPA 4: REGISTRO DE MÉTRICAS
# ============================================================================
print("\n[ETAPA 4] Registro de Métricas")
print("-"*80)

logger = task_treinamento.get_logger()

# Métricas do melhor modelo
metricas = resultado['metricas_melhor']
print("\nMetricas registradas:")
for nome, valor in metricas.items():
    if isinstance(valor, (int, float)):
        logger.report_single_value(nome, valor)
        print(f"  {nome}: {valor:.4f}")

# ============================================================================
# ETAPA 5: UPLOAD DE TABELAS E ARTEFATOS
# ============================================================================
print("\n[ETAPA 5] Upload de Tabelas e Artefatos")
print("-"*80)

# Tabela de comparação de modelos
tabela_comparacao = resultado['tabela_comparacao']
task_treinamento.upload_artifact(
    "comparacao_modelos",
    artifact_object=tabela_comparacao
)
print("Tabela de comparacao enviada como artefato")

# Criar gráfico de comparação de modelos
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))
tabela_comparacao[['MAE', 'MSE', 'RMSE', 'R2']].plot(kind='bar', ax=ax)
ax.set_title('Comparacao de Modelos - Metricas')
ax.set_xlabel('Modelos')
ax.set_ylabel('Valor')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Enviar gráfico
logger.report_matplotlib_figure(
    title="Comparacao de Modelos",
    series="Metricas",
    figure=fig,
    iteration=0
)
plt.close()
print("Grafico de comparacao enviado")

# ============================================================================
# ETAPA 6: REGISTRO DO MODELO
# ============================================================================
print("\n[ETAPA 6] Registro do Modelo")
print("-"*80)

# Criar OutputModel
melhor_modelo_nome = str(resultado['tabela_comparacao'].index[0])
output_model = OutputModel(
    task=task_treinamento,
    name=f"modelo_{melhor_modelo_nome}_conforto_termico",
    framework="PyCaret"
)

# Registrar informações do modelo
output_model.update_labels({
    "tipo": "regressao",
    "coluna_alvo": coluna_alvo,
    "melhor_modelo": melhor_modelo_nome
})

# Adicionar métricas ao modelo
output_model.update_design(config_dict={
    "metricas": {k: float(v) for k, v in metricas.items() if isinstance(v, (int, float))},
    "n_features": df_treino.shape[1] - 1,
    "n_samples_treino": df_treino.shape[0]
})

# Upload do arquivo do modelo
if 'caminho_modelo' in resultado and resultado['caminho_modelo']:
    caminho_modelo = Path(resultado['caminho_modelo'])
    if caminho_modelo.exists():
        output_model.update_weights(weights_filename=str(caminho_modelo))
        print(f"Modelo registrado: {caminho_modelo}")
    else:
        print(f"AVISO: Arquivo do modelo nao encontrado: {caminho_modelo}")

# ============================================================================
# ETAPA 7: FINALIZAÇÃO E RESUMO
# ============================================================================
print("\n[ETAPA 7] Finalizacao")
print("-"*80)

# Tags na task
task_treinamento.add_tags([
    "producao",
    melhor_modelo_nome,
    f"r2_{metricas.get('R2', 0):.2f}".replace(".", "_")
])

task_treinamento.close()

# Limpar arquivos temporários
import shutil
if temp_path.exists():
    shutil.rmtree(temp_path)

# ============================================================================
# RESUMO FINAL
# ============================================================================
print("\n" + "="*80)
print("PIPELINE COMPLETO FINALIZADO - TUDO REGISTRADO NO CLEARML!")
print("="*80)
print(f"\nProjeto: {PROJECT_NAME}")
print(f"\nDatasets criados:")
print(f"  1. Dados brutos: {dataset_bruto.id}")
print(f"  2. Dados processados: {dataset_processado.id}")
print(f"\nTasks criadas:")
print(f"  1. Processamento: {task_processamento.id}")
print(f"  2. Treinamento: {task_treinamento.id}")
print(f"\nModelo registrado:")
print(f"  - Nome: modelo_{melhor_modelo_nome}_conforto_termico")
print(f"  - Melhor modelo: {melhor_modelo_nome}")
print(f"  - R2: {metricas.get('R2', 0):.4f}")
print(f"  - MAE: {metricas.get('MAE', 0):.4f}")
print("\nAcesse o ClearML UI para visualizar todos os resultados!")
print("="*80)

## Visualizar Resultados no ClearML

Após executar a célula acima, acesse a interface web do ClearML:
- **Local**: http://localhost:8080
- **Demo**: https://app.clear.ml

### O que você vai encontrar:

**📊 No Projeto "conforto_termico":**
- 2 Tasks (Processamento + Treinamento)
- 2 Datasets (Bruto + Processado) com genealogia
- 1 Modelo registrado com métricas

**📈 Métricas rastreadas:**
- Todas as métricas de regressão (MAE, MSE, RMSE, R2, etc.)
- Comparação entre modelos
- Gráficos de comparação

**📦 Artefatos salvos:**
- Tabela de comparação de modelos
- Sample dos dados processados
- Gráfico de métricas

**🤖 Modelo:**
- Arquivo .pkl do modelo treinado
- Labels e configurações
- Genealogia completa (dados → processamento → treinamento)

## [OPCIONAL] Buscar e Reusar Recursos do ClearML

In [ ]:
# =========================================================================
# REUSAR RECURSOS DO CLEARML (Datasets, Modelos, Tasks)
# =========================================================================
import sys
sys.path.append('..')

from clearml import Task, Dataset, Model
import pandas as pd

PROJECT_NAME = "conforto_termico_def"

print("="*80)
print("BUSCANDO RECURSOS NO CLEARML")
print("="*80)

# ============================================================================
# 1. LISTAR DATASETS DISPONÍVEIS
# ============================================================================
print("\n[1] Datasets disponiveis no projeto:")
print("-"*80)

datasets = Dataset.list_datasets(
    dataset_project=PROJECT_NAME,
    partial_name="conforto_termico"
)

for i, ds in enumerate(datasets, 1):
    print(f"\n{i}. {ds.name}")
    print(f"   ID: {ds.id}")
    print(f"   Versao: {ds.version}")
    print(f"   Criado: {ds.created}")

# ============================================================================
# 2. BAIXAR DATASET ESPECÍFICO
# ============================================================================
print("\n[2] Baixando dataset processado:")
print("-"*80)

# Buscar o dataset mais recente de dados processados
dataset_processado = Dataset.get(
    dataset_project=PROJECT_NAME,
    dataset_name="dados_processados_conforto_termico"
)

# Baixar para pasta local
local_path = dataset_processado.get_local_copy()
print(f"Dataset baixado para: {local_path}")

# Carregar dados
arquivos = list(Path(local_path).glob("*.csv"))
if arquivos:
    df_downloaded = pd.read_csv(arquivos[0])
    print(f"Dados carregados: {df_downloaded.shape}")
    print(df_downloaded.head())

# ============================================================================
# 3. LISTAR TASKS DO PROJETO
# ============================================================================
print("\n[3] Tasks do projeto:")
print("-"*80)

tasks = Task.get_tasks(
    project_name=PROJECT_NAME,
    task_name=None  # Todas as tasks
)

for i, t in enumerate(tasks[:5], 1):  # Primeiras 5
    print(f"\n{i}. {t.name}")
    print(f"   ID: {t.id}")
    print(f"   Status: {t.status}")
    print(f"   Tipo: {t.task_type}")

# ============================================================================
# 4. BUSCAR MODELOS REGISTRADOS
# ============================================================================
print("\n[4] Modelos registrados:")
print("-"*80)

modelos = Model.query_models(
    project_name=PROJECT_NAME,
    model_name=None  # Todos os modelos
)

for i, modelo in enumerate(modelos, 1):
    print(f"\n{i}. {modelo.name}")
    print(f"   ID: {modelo.id}")
    print(f"   Framework: {modelo.framework}")
    print(f"   Labels: {modelo.labels}")
    
    # Baixar modelo se quiser
    # modelo_local = modelo.get_local_copy()
    # print(f"   Download: {modelo_local}")

# ============================================================================
# 5. CLONAR E REUSAR TASK
# ============================================================================
print("\n[5] Exemplo: Clonar uma task existente")
print("-"*80)

if tasks:
    task_original = tasks[0]
    print(f"Clonando task: {task_original.name}")
    
    # Clonar task (cria uma cópia)
    task_clonada = Task.clone(
        source_task=task_original.id,
        name=f"{task_original.name}_clonada",
        project=PROJECT_NAME
    )
    
    print(f"Task clonada criada: {task_clonada.id}")
    print("Você pode executar esta task clonada com configuracoes diferentes!")

print("\n" + "="*80)
print("BUSCA DE RECURSOS CONCLUIDA")
print("="*80)

---

## 📋 Resumo: ClearML vs Local

| Recurso | **Local (sem ClearML)** | **Com ClearML** |
|---------|------------------------|-----------------|
| **Execução** | Rápida, sem overhead | Overhead de registro, mas rastreável |
| **Datasets** | Arquivos locais | Versionados no servidor, com genealogia |
| **Modelos** | Apenas arquivos .pkl | Registrados com metadados, métricas, tags |
| **Métricas** | Apenas print no console | Dashboard visual, comparável |
| **Reprodutibilidade** | Manual (código) | Automática (parâmetros, ambiente, código) |
| **Colaboração** | Difícil | Fácil (servidor compartilhado) |
| **Histórico** | Não rastreado | Completo (todas as execuções) |

### 💡 Quando usar cada um:

**Use LOCAL (célula anterior):**
- Desenvolvimento rápido
- Testes locais
- Notebooks exploratórios
- Quando não tem servidor ClearML

**Use ClearML (esta seção):**
- Produção
- Experimentos para comparar
- Trabalho em equipe
- Quando precisa rastrear tudo
- Deploy de modelos

## Pipeline Completo COM Decorators (@PipelineDecorator)

Esta é a solução **completa** usando os decorators oficiais do ClearML:
- `@PipelineDecorator.pipeline` - Define o pipeline completo
- `@PipelineDecorator.component` - Define cada componente do pipeline

### 🎯 O que este pipeline faz:

1. **Cria projeto automaticamente** no ClearML (se não existir)
2. **Registra 3 datasets versionados**:
   - Dados brutos
   - Dados processados
   - Dados com features
3. **Cria 5 tasks separadas** (uma para cada componente)
4. **Registra modelo** com métrica principal (R²) para decisão de modelo vigente
5. **Artefatos**: tabelas, gráficos, encoders, scalers

### 📝 Modos de execução:

- **Local**: `run_locally=True` - executa no notebook, mas registra tudo no ClearML
- **Remoto**: `run_locally=False` - envia para fila de execução remota

### ⚙️ Arquivo criado:

[src/clearml/pipeline_completo_decorators.py](../src/clearml/pipeline_completo_decorators.py)

---

### ⚠️ IMPORTANTE: Problema de Cache

Se você receber o erro `OSError: could not get source code`:

**SOLUÇÃO**: Reinicie o kernel do notebook:
1. Pressione `Ctrl+Shift+P` (ou `Cmd+Shift+P` no Mac)
2. Digite "Restart Kernel"
3. Selecione "Jupyter: Restart Kernel"
4. Execute a célula abaixo novamente

Isso acontece porque o Python mantém em cache o código fonte dos módulos importados. Quando modificamos o arquivo depois de importá-lo, o cache fica desatualizado.

In [ ]:
# ============================================================================
# IMPORTANTE: EXECUTE ESTA CELULA APENAS UMA VEZ
# Se der erro de "could not get source code", reinicie o kernel manualmente:
# Ctrl+Shift+P -> "Restart Kernel" e execute esta celula novamente
# ============================================================================

from src.clearml.pipeline_completo_decorators import executar_pipeline
from pathlib import Path

# Usar caminho absoluto para evitar problemas
CAMINHO_DADOS = str(Path('../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv').resolve())

print("="*80)
print("EXECUTANDO PIPELINE COMPLETO COM @PipelineDecorator")
print("="*80)
print(f"\nArquivo CSV: {CAMINHO_DADOS}")
print(f"Arquivo existe: {Path(CAMINHO_DADOS).exists()}")
print("Modo: EXECUCAO LOCAL (com tracking ClearML)\n")
print("="*80)

try:
    resultado = executar_pipeline(
        caminho_csv=CAMINHO_DADOS,
        run_locally=True  # True = local, False = remoto (precisa de fila configurada)
    )
    
    print("\n" + "="*80)
    print("RESUMO FINAL")
    print("="*80)
    
    if resultado:
        print(f"\nPipeline concluido com sucesso!")
        print(f"\nDatasets criados:")
        for key, value in resultado.items():
            if 'dataset' in key:
                print(f"  - {key}: {value}")
        
        print(f"\nModelo:")
        if 'model' in resultado:
            print(f"  - ID: {resultado['model']}")
        
        print(f"\nMetricas:")
        if 'metricas' in resultado:
            for metric, value in resultado['metricas'].items():
                if isinstance(value, (int, float)):
                    print(f"  - {metric}: {value:.4f}")
    else:
        print("\nPipeline falhou")
        
except Exception as e:
    print("\n" + "="*80)
    print("ERRO NO PIPELINE")
    print("="*80)
    print(f"\nErro: {type(e).__name__}")
    print(f"Mensagem: {str(e)}")
    import traceback
    print("\nTraceback completo:")
    traceback.print_exc()

print("\n" + "="*80)

### 🔍 Como funciona internamente:

O pipeline usa **5 componentes** encadeados:

```python
@PipelineDecorator.component - component_upload_dados_brutos
    ↓ (dataset_bruto_id)
@PipelineDecorator.component - component_pipeline_processamento
    ↓ (dataset_processado_id)
@PipelineDecorator.component - component_pipeline_features
    ↓ (dataset_features_id)
@PipelineDecorator.component - component_pipeline_treinamento
    ↓ (resultado_treinamento, caminho_modelo)
@PipelineDecorator.component - component_registrar_modelo
    ↓ (model_id)
@PipelineDecorator.pipeline - pipeline_completo_clearml
    ↓ (resumo completo)
```

Cada componente:
- Cria sua própria Task no ClearML
- Pode ser executado independentemente
- Pode ter cache (exceto treinamento)
- Retorna valores serializáveis para o próximo

### 📌 Métrica Principal para Decisão de Modelo Vigente

O pipeline registra **R²** como métrica principal no modelo. Esta métrica é usada para:
- Comparar modelos entre experimentos
- Decidir qual modelo deve ser promovido para produção
- Configurável em `METRICA_PRINCIPAL` no arquivo

Para classificação, você pode mudar para: `METRICA_PRINCIPAL = "Accuracy"` ou `"AUC"`

In [ ]:
from src.clearml.executar_pipelines import executar_pipelines_completo

resultado = executar_pipelines_completo(
    caminho_csv="dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv",
    coluna_alvo="p1",
    n_modelos=3
)

In [ ]:
# ============================================================================
# DIAGNÓSTICO: Verificar Configuração Antes de Executar Pipelines
# ============================================================================

import sys
from pathlib import Path

print("="*80)
print("DIAGNÓSTICO PRÉ-PIPELINE")
print("="*80)

# 1. Verificar arquivo de dados
arquivo_dados = Path('../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv')
print(f"\n1. Arquivo de dados:")
print(f"   Caminho: {arquivo_dados.resolve()}")
print(f"   Existe: {arquivo_dados.exists()}")
if arquivo_dados.exists():
    print(f"   Tamanho: {arquivo_dados.stat().st_size / 1024:.2f} KB")

# 2. Verificar configuração ClearML
print(f"\n2. ClearML:")
try:
    from clearml import Task
    # Tentar criar uma task de teste (sem inicializar realmente)
    print(f"   ✓ ClearML instalado")
    
    # Verificar arquivo de config
    clearml_conf = Path.home() / 'clearml.conf'
    print(f"   Config: {clearml_conf}")
    print(f"   Config existe: {clearml_conf.exists()}")
    
except ImportError as e:
    print(f"   ✗ ClearML não instalado: {e}")
except Exception as e:
    print(f"   ⚠ Aviso: {e}")

# 3. Verificar módulos do projeto
print(f"\n3. Módulos do projeto:")
try:
    from src.clearml.pipeline_01_processamento import pipeline_processamento
    print(f"   ✓ pipeline_01_processamento importado")
except ImportError as e:
    print(f"   ✗ Erro ao importar pipeline_01_processamento: {e}")

try:
    from src.pipelines.pipeline_processamento import executar_pipeline_processamento
    print(f"   ✓ executar_pipeline_processamento importado")
except ImportError as e:
    print(f"   ✗ Erro ao importar executar_pipeline_processamento: {e}")

# 4. Verificar Python path
print(f"\n4. Python path:")
print(f"   Diretório atual: {Path.cwd()}")
print(f"   '..' no sys.path: {'..' in sys.path}")

print("\n" + "="*80)
print("Se tudo estiver ✓, você pode executar os pipelines ClearML")
print("Se houver ✗, corrija os problemas antes de continuar")
print("="*80)

### 🔧 Solução Aplicada: PipelineDecorator vs Task.init()

#### ❌ **Problema Identificado**: `ValueError: Pipeline step "processar_dados" ... failed`

**Causa Real**: O uso de `@PipelineDecorator` cria tasks que são **enviadas para execução remota** no servidor ClearML, mesmo quando você executa localmente. Essas tasks remotas falham porque:

1. ❌ O arquivo CSV está na sua máquina, não no servidor
2. ❌ O ambiente remoto pode não ter as dependências instaladas
3. ❌ O código das suas funções não é transferido corretamente

#### ✅ **Solução Implementada**: Usar `Task.init()` diretamente

Mudamos de:
```python
# ❌ PROBLEMÁTICO - cria tasks remotas
@PipelineDecorator.pipeline(...)
@PipelineDecorator.component(...)
def processar_dados(...):
    ...
```

Para:
```python
# ✅ FUNCIONA - executa localmente com tracking
def pipeline_processamento(...):
    task = Task.init(...)  # Cria task local
    # ... seu código aqui ...
    task.close()
```

**Vantagens**:
- ✅ Executa **localmente** (acessa seus arquivos CSV)
- ✅ **Registra tudo** no servidor ClearML (datasets, métricas, artefatos)
- ✅ Não precisa configurar agentes remotos
- ✅ Mais controle sobre a execução

**Quando usar cada abordagem**:
- `Task.init()` → Desenvolvimento, notebooks, execução local com tracking
- `@PipelineDecorator` → Produção com agentes ClearML configurados e dados no servidor

In [ ]:
# ============================================================================
# Pipeline 1: Processamento de Dados (VERSÃO CORRIGIDA)
# ============================================================================
# MUDANÇA: Agora usa Task.init() em vez de @PipelineDecorator
# MOTIVO: Decorators enviam tasks para servidor remoto, causando falhas
# SOLUÇÃO: Execução local com tracking ClearML completo
# ============================================================================

from src.clearml.pipeline_01_processamento import pipeline_processamento

# Caminho do arquivo (relativo ao notebook)
arquivo = '../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv'

print("="*80)
print("Pipeline 1: Processamento de Dados")
print("Modo: EXECUÇÃO LOCAL com tracking ClearML")
print("="*80)
print(f"Arquivo: {arquivo}\n")

try:
    # Executar pipeline de processamento
    # Agora executa LOCALMENTE e registra no ClearML
    r1 = pipeline_processamento(arquivo)
    
    print("\n✓ Pipeline 1 concluído com sucesso!")
    print(f"\n📦 Resultados:")
    print(f"   Dataset ID: {r1['dataset_processado_id']}")
    print(f"   Shape: {r1['shape']}")
    print(f"   NAs removidos: {r1['nas_removidos']}")
    print("\n✓ Acesse o ClearML UI para ver métricas, dataset e artefatos!")
    print("="*80)
    
except Exception as e:
    print(f"\n✗ Erro no Pipeline 1:")
    print(f"  Tipo: {type(e).__name__}")
    print(f"  Mensagem: {str(e)}")
    print("\n💡 Dicas:")
    print("  • Verifique se o arquivo CSV existe")
    print("  • Verifique a conexão com o servidor ClearML")
    print("  • Veja os logs completos acima para mais detalhes")
    raise

Pipeline 1: Processamento de Dados
Modo: EXECUÇÃO LOCAL com tracking ClearML
Arquivo: ../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv



2026-02-09 19:06:37,331 - WARNING - Retrying (Retry(total=237, connect=240, read=237, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /v2.23/projects.get_all
2026-02-09 19:07:04,628 - WARNING - Retrying (Retry(total=236, connect=240, read=236, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /v2.23/projects.get_all
2026-02-09 19:07:39,962 - WARNING - Retrying (Retry(total=235, connect=240, read=235, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /v2.23/projects.get_all
2026-02-09 19:08:31,254 - WARNING - Retrying (Retry(total=234, connect=240, read=234, redirect=240, status=240)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /v2.23/projects.get_all


In [ ]:

# Pipeline 2 (usa ID do pipeline 1)
from src.clearml.pipeline_02_features import pipeline_features
r2 = pipeline_features(dataset_processado_id=r1['dataset_processado_id'])

# Pipeline 3 (usa ID do pipeline 2)
from src.clearml.pipeline_03_treinamento import pipeline_treinamento
r3 = pipeline_treinamento(dataset_features_id=r2['dataset_features_id'])